In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
#import plotly as plt
import pickle
import logging

C:\Users\magnusxd\AppData\Local\Temp\ipykernel_23452\619512384.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:
# load data set from xls

all_df = pd.read_csv('D:\\Programming\\PycharmProjects\\ba\\imbalanced_data\\data\\stroke\\00_raw\\healthcare-dataset-stroke-data.xls',na_values=["na"])

In [17]:
all_df.shape

(5110, 12)

AttributeError: 'Series' object has no attribute 'unique_value'

In [3]:
all_df.head()

# preprocessing change gender, ever_married to -> 0, 1
# one hot encode work_type, residence_type, smoking status

#drop or and impute ???? track how many missing field rows 

# scale


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
unique_values_test = all_df["stroke"].value_counts()
pos_class = unique_values_test.index[0]
neg_class = unique_values_test.index[1]
pos_count = unique_values_test.iloc[0]
neg_count = unique_values_test.iloc[1]
print("{} value count: {}".format(pos_class, pos_count))
print("{} value count: {}".format(neg_class, neg_count))
print("{} percentage: {}".format(pos_class, '%.2f'%(pos_count/(pos_count + neg_count))))
print("{} percentage: {}".format(pos_class, '%.2f'%(neg_count/(pos_count + neg_count))))
print("{} - {} ratio: {}".format(pos_class, neg_class, "1 : {}".format(pos_count/neg_count)))
print("{} - {} ratio: {}".format(pos_class, neg_class, "3 : {}".format((pos_count/neg_count)*3)))

0 value count: 4861
1 value count: 249
0 percentage: 0.95
0 percentage: 0.05
0 - 1 ratio: 1 : 19.522088353413654
0 - 1 ratio: 3 : 58.566265060240966


In [6]:
all_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,5110.0,36517.829354,21161.721625,67.00,17741.250,36932.000,54682.00,72940.00
age,5110.0,43.226614,22.612647,0.08,25.000,45.000,61.00,82.00
hypertension,5110.0,0.097456,0.296607,0.00,0.000,0.000,0.00,1.00
heart_disease,5110.0,0.054012,0.226063,0.00,0.000,0.000,0.00,1.00
avg_glucose_level,5110.0,106.147677,45.283560,55.12,77.245,91.885,114.09,271.74
bmi,4909.0,28.893237,7.854067,10.30,23.500,28.100,33.10,97.60
stroke,5110.0,0.048728,0.215320,0.00,0.000,0.000,0.00,1.00


In [3]:
nan_values = all_df.isna().sum(axis=0).sort_values(ascending=False)

In [16]:
nan_values.head

<bound method NDFrame.head of bmi                  201
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
smoking_status         0
stroke                 0
dtype: int64>

In [28]:
print("Percentage NaN Values (BMI colum) %.2f"%((nan_values["bmi"]/all_df.shape[0])*100))

Percentage NaN Values (BMI colum) 3.93


In [20]:
unique_values_test = all_df.dropna()["stroke"].value_counts()
pos_class = unique_values_test.index[0]
neg_class = unique_values_test.index[1]
pos_count = unique_values_test.iloc[0]
neg_count = unique_values_test.iloc[1]
print("{} value count: {}".format(pos_class, pos_count))
print("{} value count: {}".format(neg_class, neg_count))
print("{} percentage: {}".format(pos_class, '%.2f'%(pos_count/(pos_count + neg_count))))
print("{} percentage: {}".format(pos_class, '%.2f'%(neg_count/(pos_count + neg_count))))
print("{} - {} ratio: {}".format(pos_class, neg_class, "1 : {}".format(pos_count/neg_count)))
print("{} - {} ratio: {}".format(pos_class, neg_class, "3 : {}".format((pos_count/neg_count)*3)))

0 value count: 4700
1 value count: 209
0 percentage: 0.96
0 percentage: 0.04
0 - 1 ratio: 1 : 22.48803827751196
0 - 1 ratio: 3 : 67.46411483253588


In [4]:
print(all_df.shape)

# drop rows with NaN bmi values (set index labels to 0, 1, ... n-1 to prevent issues when concatenating one hot encoded discrete features back to array later
all_df = all_df.dropna(ignore_index=True)
print(all_df.shape)
# drop id row
all_df = all_df.drop(labels=["id"], axis=1)

print(all_df.shape)


(5110, 12)
(4909, 12)
(4909, 11)


In [129]:
#pd.DataFrame([1,2,3,np.NAN,5,6], columns=["number"]).dropna(ignore_index=True)

,number
0,1.0
1,2.0
2,3.0
3,5.0
4,6.0


In [5]:
from sklearn.preprocessing import OneHotEncoder

#create categories list from unique row values

discrete_columns = all_df.dtypes[all_df.dtypes=='object'].index.to_list()
print(f"\ndiscrete columns: {discrete_columns}")

unique_values_count = all_df[discrete_columns].nunique(axis=0)
print("\nunique value count for discrete columns:\n{}".format(unique_values_count))
print("\ntotal rows one hot encoded: {}".format(unique_values_count.sum()))

categories = []
print("\nlist unique values of discrete columns:")
for column in discrete_columns:
    unique_values = pd.unique(all_df[column]).tolist()
    categories.append(unique_values)
    print(unique_values)

flat_categories = [unq_val for uniq_vals in categories for unq_val in uniq_vals]
print("\nflat categories for pd column names: {}".format(flat_categories))

enc = OneHotEncoder(sparse_output=False, categories=categories)
enc.fit(all_df[discrete_columns])
result = enc.transform(all_df[discrete_columns])
# print(f"({len(result)},{len(result[0])})")
one_hot_encoded_discrete_features_df = pd.DataFrame(result, columns=flat_categories)
print(one_hot_encoded_discrete_features_df.shape)
print(one_hot_encoded_discrete_features_df.columns)



# print(type(result))
# print(len(result[0]))
# print(enc.categories_)
# print(enc.n_features_in_)
# print(enc.get_feature_names_out())


discrete columns: ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

unique value count for discrete columns:
gender            3
ever_married      2
work_type         5
Residence_type    2
smoking_status    4
dtype: int64

total rows one hot encoded: 16

list unique values of discrete columns:
['Male', 'Female', 'Other']
['Yes', 'No']
['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked']
['Urban', 'Rural']
['formerly smoked', 'never smoked', 'smokes', 'Unknown']

flat categories for pd column names: ['Male', 'Female', 'Other', 'Yes', 'No', 'Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked', 'Urban', 'Rural', 'formerly smoked', 'never smoked', 'smokes', 'Unknown']
(4909, 16)
Index(['Male', 'Female', 'Other', 'Yes', 'No', 'Private', 'Self-employed',
       'Govt_job', 'children', 'Never_worked', 'Urban', 'Rural',
       'formerly smoked', 'never smoked', 'smokes', 'Unknown'],
      dtype='object')


In [7]:
# drop old discrete colums
all_df = all_df.drop(labels=discrete_columns, axis=1)
print(all_df.columns)
print(all_df.shape)
# print(np.sum(all_df.index.to_numpy()))

print(one_hot_encoded_discrete_features_df.columns)
print(one_hot_encoded_discrete_features_df.shape)
# print(np.sum(one_hot_encoded_discrete_features_df.index.to_numpy()))



# add on hot encoded discrete columns to df
new_df = pd.concat([all_df, one_hot_encoded_discrete_features_df], axis=1)
# new_df = all_df.append(one_hot_encoded_discrete_features_df)
print(new_df.shape)

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'stroke'],
      dtype='object')
(4909, 6)
12046686
Index(['Male', 'Female', 'Other', 'Yes', 'No', 'Private', 'Self-employed',
       'Govt_job', 'children', 'Never_worked', 'Urban', 'Rural',
       'formerly smoked', 'never smoked', 'smokes', 'Unknown'],
      dtype='object')
(4909, 16)
12046686
(4909, 22)


In [60]:
# from sklearn.preprocessing import OneHotEncoder
# categories = [['female'],['from Europe', 'from US'],['uses Chrome', 'uses Edge', 'uses Firefox', 'uses Safari']]
# enc = OneHotEncoder(sparse_output=False, categories=categories)
# X = [['female', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox'],['female', 'from Europe', 'uses Chrome'],['female', 'from Europe', 'uses Edge']]
# enc = enc.fit(X)
# result = enc.transform([['female', 'from US', 'uses Safari'],['female', 'from Europe', 'uses Firefox'],['female', 'from Europe', 'uses Chrome'],['female', 'from Europe', 'uses Edge']])
# print(result)
# print(type(result))
# print(len(result[0]))
# print(enc.categories_)
# print(enc.n_features_in_)
# print(enc.get_feature_names_out())


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [16]:
print(new_df.head())
print(new_df.shape)


    age  hypertension  heart_disease  avg_glucose_level   bmi  stroke  Male  \
0  67.0             0              1             228.69  36.6       1   1.0   
1  80.0             0              1             105.92  32.5       1   1.0   
2  49.0             0              0             171.23  34.4       1   0.0   
3  79.0             1              0             174.12  24.0       1   0.0   
4  81.0             0              0             186.21  29.0       1   1.0   

   Female  Other  Yes  ...  Self-employed  Govt_job  children  Never_worked  \
0     0.0    0.0  1.0  ...            0.0       0.0       0.0           0.0   
1     0.0    0.0  1.0  ...            0.0       0.0       0.0           0.0   
2     1.0    0.0  1.0  ...            0.0       0.0       0.0           0.0   
3     1.0    0.0  1.0  ...            1.0       0.0       0.0           0.0   
4     0.0    0.0  1.0  ...            0.0       0.0       0.0           0.0   

   Urban  Rural  formerly smoked  never smoked  sm

In [15]:
print(all_df.head())

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  
